
#### Intro
This example shows you how to use Approximate Nearest Neighbor Search (ANN) using [Hnswlib](https://github.com/nmslib/hnswlib/).

Install it with: `pip install hnswlib` (along with other requirements)

The embeddings model is `msmarco-distilbert-base-v4` from [SBERT](https://sbert.net/examples/applications/semantic-search/README.html)

The dataset we use is our own, (at ../dish-washer-data.csv); that gives us a way to compare with naive search



In [ ]:
!pip install hnswlib
!pip install pandas
!pip3 install torch
!pip install sentence_transformers

In [ ]:
import os
import csv
import pickle
import time
import hnswlib
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
import numpy as np

Your configuration below

In [ ]:
model_name = 'msmarco-distilbert-base-v4'
embeddings_model = SentenceTransformer(model_name)
embedding_size = 768    #Size of embeddings
top_k_hits = 5         #Output k hits

Helper functions to get the embeddings from the embeddings model

In [ ]:
def get_embedding(text: str, model: SentenceTransformer) -> list[float]:
    embeddings = model.encode([text])
    return embeddings

def get_doc_embedding(text: str, model: SentenceTransformer) -> list[float]:
    return get_embedding(text, model)

def get_query_embedding(text: str, model: SentenceTransformer) -> list[float]:
    return get_embedding(text, model)

def compute_doc_embeddings(df: pd.DataFrame, model: SentenceTransformer) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe.
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_doc_embedding(r.content.replace("\n", " "), model) for idx, r in df.iterrows()
    }

If this is the first run, generate the embeddings, if not, then load the previously generated embeddings 

In [ ]:
embeddings_cache_path = f"dishwasher-repair-manual-embeddings-{model_name.replace('/', '_')}.pkl"

#Check if embeddings cache path exists
if not os.path.exists(embeddings_cache_path):
    df = pd.read_csv('../dish-washer-data.csv')
    df["tokens"] = pd.to_numeric(df["tokens"])  # convert column "tokens" of a DataFrame
    df = df.set_index(["title", "heading"])
    print(f"{len(df)} rows in the data.")
    print(df.sample(10))
    # TODO get some stats on max/min content length
    corpus_size = df.shape[0]

    # This could take a bit of time
    print("Encoding the corpus. This might take a while...")
    corpus_embeddings = compute_doc_embeddings(df, embeddings_model)

    print("Store file...")
    with open(embeddings_cache_path, "wb") as fOut:
        pickle.dump({'sentences': df, 'embeddings': corpus_embeddings}, fOut)
else:
    print("Loading pre-computed embeddings from disc...")
    with open(embeddings_cache_path, "rb") as f_in:
        cache_data = pickle.load(f_in)
        corpus_sentences = cache_data['sentences']
        corpus_embeddings = cache_data['embeddings']
        corpus_embeddings = [v for k, v in corpus_embeddings.items()]
        print("Loading done.")
        print(f'Length, corpus sentences:{len(corpus_sentences)}')
        print(f'Length, corpus embeddings:{len(corpus_embeddings)}')
        print(f'one vector dimension: {len(corpus_embeddings[0][0])}')

        print(f"Sentence samples\n {corpus_sentences.sample(5)}")
        


In [ ]:
#Defining our hnswlib index
index_path = "./hnswlib.index"
index = hnswlib.Index(space = 'cosine', dim = embedding_size)

if os.path.exists(index_path):
    print("Loading index...")
    index.load_index(index_path)
    print('Loading index done.')
else:
    ### Create the HNSWLIB index
    print("Start creating HNSWLIB index")
    # TODO check is ef_consturction and M and appropriate for this dataset
    index.init_index(max_elements = len(corpus_embeddings), ef_construction = 400, M = 64)

    # create ids for the embeddings, ids are optional N-size numpy array of integer labels for all elements in data. 
    ids = np.arange(len(corpus_embeddings))
    print(f'ids:\n{ids}')
    
    # Then we train the index to find a suitable clustering
    corpus_embeddings = np.array(corpus_embeddings).squeeze()
    print('shape', corpus_embeddings.shape)
    print(f'corpus embeddings:\n{corpus_embeddings}')
    index.add_items(corpus_embeddings, ids)
    # ?? index.add_items(corpus_embeddings, list(corpus_sentences.index))

    print("Saving index to:", index_path)
    index.save_index(index_path)

# Controlling the recall by setting ef:
index.set_ef(50)  # ef should always be > top_k_hits



In [ ]:
# Choose questions
# inp_question = "Why is my dishwasher leaking?"
inp_question = "Why do we use a dishwasher?"

In [ ]:
start_time = time.time()
question_embedding = embeddings_model.encode(inp_question)

# Use hnswlib knn_query method to find the top_k_hits
corpus_ids, distances = index.knn_query(question_embedding, k=top_k_hits)

# Extract corpus ids and scores for the query
hits = [{'corpus_id': id, 'score': 1 - score} for id, score in zip(corpus_ids[0], distances[0])]
hits = sorted(hits, key=lambda x: x['score'], reverse=True)
end_time = time.time()
print(f'Hits\n, {hits}')

print("Input question:", inp_question)
print("Results (after {:.3f} seconds):".format(end_time-start_time))
for hit in hits[0:top_k_hits]:
    print(f"\t{hit['score']:.3f}\t{corpus_sentences.iloc[hit['corpus_id']].content}")
    



In [ ]:
# Approximate Nearest Neighbor (ANN) is not exact, it might miss entries with high cosine similarity
# Here, we compute the recall of ANN compared to the exact results
correct_hits = util.semantic_search(torch.tensor(question_embedding), 
                                    torch.tensor(corpus_embeddings).squeeze(), 
                                    top_k=top_k_hits)[0]

print(f'Correct hits:\n {correct_hits}')

for hit in correct_hits[0:top_k_hits]:
    print(f"\t{hit['score']:.3f}\t{corpus_sentences.iloc[hit['corpus_id']].content}")

correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])

ann_corpus_ids = set([hit['corpus_id'] for hit in hits])
if len(ann_corpus_ids) != len(correct_hits_ids):
    print("Approximate Nearest Neighbor returned a different number of results than expected")

recall = len(ann_corpus_ids.intersection(correct_hits_ids)) / len(correct_hits_ids)
print("\nApproximate Nearest Neighbor Recall@{}: {:.2f}".format(top_k_hits, recall * 100))

if recall < 1:
    print("Missing results:")
    for hit in correct_hits[0:top_k_hits]:
        if hit['corpus_id'] not in ann_corpus_ids:
            print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))
print("\n\n========\n")
